In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
import numpy as np
import os
import time
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\train'
validation_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\val'
nb_train_samples = 50041
nb_validation_samples = 12117
epochs = 100
batch_size = 16
img_channels = 3
nb_classes = 3

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])

label_map = (train_generator.class_indices)
print(label_map)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])


# checkpoint
current_directory = os.getcwd()

checkpoint_dir = os.path.join(current_directory, 'checkpoints')
result_dir = os.path.join(current_directory, 'results')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

Using TensorFlow backend.


Found 50041 images belonging to 3 classes.
{'down': 0, 'medium': 1, 'up': 2}
Found 12117 images belonging to 3 classes.


### Load base model

In [2]:
from keras.models import load_model    

model_file = 'model_nasnetmobile_view-offset-altitude_08.hdf5'
model_full_path = os.path.join(result_dir, model_file)
model = load_model(model_full_path)

model.summary()

# weights = model.get_weights()
# print(len(weights))
# print(len(model.layers))
# print(len(weights[:-4]))
# for i in range(0, len(weights)):
#     print(weights[i].shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 255, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 255, 255, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 255, 255, 32) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

### Transfering to new model

In [3]:
from keras.models import Model
new_model = Model(model.inputs, model.layers[-3].output)

for i in range(0, (len(new_model.layers)-2)):
    new_model.layers[i].trainable = False
    
new_model.summary()
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

weights = model.get_weights()
new_model.set_weights(weights[:-4])

# delete(model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 255, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 255, 255, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 255, 255, 32) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

reduction_concat_reduce_4 (Conc (None, 32, 32, 352)  0           reduction_add_2_reduce_4[0][0]   
                                                                 reduction_add3_reduce_4[0][0]    
                                                                 add_3[0][0]                      
                                                                 reduction_add4_reduce_4[0][0]    
__________________________________________________________________________________________________
adjust_avg_pool_1_5 (AveragePoo (None, 32, 32, 264)  0           adjust_relu_1_5[0][0]            
__________________________________________________________________________________________________
adjust_avg_pool_2_5 (AveragePoo (None, 32, 32, 264)  0           cropping2d_3[0][0]               
__________________________________________________________________________________________________
adjust_conv_1_5 (Conv2D)        (None, 32, 32, 44)   11616       adjust_avg_pool_1_5[0][0]        
__________

In [4]:
#check if weights are loaded into the new_model
new_model.get_weights()

[array([[[[-1.54771611e-01,  1.48094028e-01,  2.15805888e-01,
           -8.11043754e-02, -3.84758890e-01,  4.29857343e-01,
            1.32616132e-01,  7.63416886e-01, -9.11567658e-02,
           -1.19143558e+00, -4.60367411e-01, -1.77604273e-01,
            3.31826091e-01,  4.71088111e-01, -1.62616730e-01,
            4.58805449e-02,  1.26458332e-03, -1.88001171e-01,
           -1.33091122e-01, -2.71796972e-01,  7.77908340e-02,
            9.36157107e-01,  5.15434384e-01,  7.29162171e-02,
            2.49711394e-01, -5.54241061e-01,  1.17760621e-01,
           -8.81484300e-02, -3.27346414e-01, -5.16985059e-01,
           -5.45115888e-01,  4.16371763e-01],
          [-1.46266788e-01,  8.76469612e-02, -1.65042449e-02,
           -1.94386452e-01, -3.65291387e-01, -2.41228744e-01,
            2.51062721e-01,  5.13525188e-01,  7.57760368e-03,
           -4.92016703e-01, -1.41688064e-01, -2.79397875e-01,
           -4.15535241e-01,  1.88553736e-01,  1.92790050e-02,
            1.99668389e-

### Training the transfered-model

In [5]:
file="model_nasnetmobile_cam_pitch_transfer-learning_{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

NAME = "nasnetmobile_cam-pitch_transfer-learning_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), write_graph = True, write_images=True)

new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, tensorboard])

Epoch 1/100
3127/3127 [==============================] - 1629s 521ms/step - loss: 0.4162 - acc: 0.8926 - val_loss: 0.2885 - val_acc: 0.9127

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_nasnetmobile_cam_pitch_transfer-learning_01.hdf5
Epoch 2/100
3127/3127 [==============================] - 1765s 564ms/step - loss: 0.0907 - acc: 0.9728 - val_loss: 0.2131 - val_acc: 0.9338

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_nasnetmobile_cam_pitch_transfer-learning_02.hdf5
Epoch 3/100
3127/3127 [==============================] - 1659s 530ms/step - loss: 0.0692 - acc: 0.9784 - val_loss: 0.1936 - val_acc: 0.9409

Epoch 00003: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_nasnetmobile_cam_pitch_transfer-learning_03.hdf5
Epoch 4/100
3127/3127 [==============================] - 1773s 567ms/step - loss: 0.0592 - acc: 0.9810 - val_loss: 0.1683 - val_acc: 0.9495

E

KeyboardInterrupt: 